## Exceptions

In [3]:
answer = 92

try:
    if answer != 42:
        raise ValueError("Wrong answer")
except RuntimeError as e:
    print(f"Error occurred {e}")
    raise e
finally:
    print("Cleanup")

Cleanup


ValueError: Wrong answer

In [7]:

def outer():
    try:
        middle()
    except Exception as e:
        print(f"Exception: {e}")
        raise e

def middle():
    try:
        inner()
    finally:
        print("cleanup")

def inner():
    raise RuntimeError("Kaboom")

outer()

cleanup
Exception: Kaboom


RuntimeError: Kaboom

In [8]:
BaseException.__subclasses__()

[Exception, GeneratorExit, SystemExit, KeyboardInterrupt]

In [9]:
Exception.__subclasses__()[:5] #should catch this

[TypeError, StopAsyncIteration, StopIteration, ImportError, OSError]

In [10]:
import sys

try:
    sys.exit()
except: #bad, will not exit
    pass


try:
    sys.exit()
except Exception: # catch all
    pass

SystemExit: 

D:\Tools\Python\Anaconda\envs\python_course\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### RuntimeError

In [11]:
d = {"foo": 42, "bar": 24}

for key in d:
    d.pop(key)

RuntimeError: dictionary changed size during iteration

In [15]:
try:
    import foobar_speedups as foobar  #example : c module
except ImportError:
    print("import error")
    # import foobar_slow as foobar  #example: python module

import error


### AttributeError

In [16]:
class A:
    pass
A().foobar

AttributeError: 'A' object has no attribute 'foobar'

### LookupError (IndexError, KeyError)

### TypeError
### ValueError

### Own Exceptions

In [17]:
class Error(Exception):
    """Exception that is the base class of all other error exceptions.
    You can use this to catch all errors with one single except statement
    """
    pass

class DatabaseError(Error):
    """Exception that are related to database.
    """
    pass

class InterfaceError(Error):
    pass

e = Exception("hello", 92, "world")
print(e.args)

('hello', 92, 'world')


In [18]:
raise e

Exception: ('hello', 92, 'world')

In [19]:
e.__traceback__  # list of stack frames

In [21]:
#traceback will be attached by raise operator
e2 = Exception()
print(e.__traceback__ is None)

e3 = e2.with_traceback(e.__traceback__)
print(e3 is e2 and e3.__traceback__ is not None)

False
True


#### How to print traceback

In [23]:
import traceback

def foo():
    bar()

def bar():
    raise Exception

try:
    foo()
except Exception as e:
    traceback.print_tb(e.__traceback__)

  File "<ipython-input-23-9f660f2ff1a9>", line 10, in <module>
    foo()
  File "<ipython-input-23-9f660f2ff1a9>", line 4, in foo
    bar()
  File "<ipython-input-23-9f660f2ff1a9>", line 7, in bar
    raise Exception


### Exceptions -- Causes

In [27]:
class LibraryError(Exception):
    pass

try:
    open("I_don't_exist.rly")
except OSError as e:
    raise LibraryError

## exception will be in e.__context__

LibraryError: 

Better way to do the same

In [28]:

try:
    open("I_don't_exist.rly")
except OSError as e:
    raise LibraryError from e

## exception will be in e.__cause__

LibraryError: 

In [30]:
#raise #re-raise last exception

## Else

In [31]:
try:
    open("example.txt")
    #something
except IOError as e:
    print(e, file = sys.stderr)
else:
    print("success")

[Errno 2] No such file or directory: 'example.txt'


## with

In [32]:
with open("file1.db") as db:
    with open("file2.db") as db2:
        pass;
    #calls db2.close() automatically
#calls db.close automatically

FileNotFoundError: [Errno 2] No such file or directory: 'file1.db'

with is used with context managers

In [36]:
from contextlib import AbstractContextManager # >= 3.6
from functools import partial

class oppened(AbstractContextManager):


    def __init__(self, path, *args, **kwargs)-> None:
        self.oppener = partial(open, path, *args, **kwargs)
    def __enter__(self):
        self.file = self.oppener()
        return self.file
    def __exit__(self, exc_type, exc_value, traceback):
        self.file.close()
        del self.file

manager1 = oppened("input.txt")
manager2 = oppened("output.txt")
with manager1 as f:
    with manager2 as g:
        pass

### one more context manager

In [40]:
import os

class cd(AbstractContextManager):
    def __init__(self, target):
        self.target = target
    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.target)
    def __exit__(self, exc_type, exc_val, exc_tb):
        os.chdir(self.saved_cwd)
        del self.saved_cwd  # restrict double entering


print(os.getcwd())
with cd("dummy_data"):
    print(os.getcwd())
print(os.getcwd())

D:\Projects\PyCharm Projects\Python_cs_center
D:\Projects\PyCharm Projects\Python_cs_center\dummy_data
D:\Projects\PyCharm Projects\Python_cs_center


### one more context manager. temp file

In [45]:
import tempfile

with tempfile.NamedTemporaryFile() as file:
    path = file.name

C:\Users\Denis\AppData\Local\Temp\tmpg83heb9v


### one more context manager. redirect stdout

In [51]:
import io
from contextlib import redirect_stdout

file = io.StringIO()
with redirect_stdout(file):
    print("Hello, world!")
assert file.getvalue() == "Hello, world!\n"

### one more context manager. suppress exception

In [52]:
import os
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove("non-existing-file.txt")

### Contextlib. ContextDecorator
Mixin that converts the ContextManager to decorator

In [54]:
from contextlib import suppress, ContextDecorator

class suppressed(suppress, ContextDecorator):
    pass

@suppressed(IOError)
def do_something():
    pass